In [1]:
#SPATIU DE IMPORT-URI
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QLineEdit, QPushButton, 
                             QTextEdit, QVBoxLayout, QHBoxLayout, QMessageBox, QFileDialog, QComboBox, QInputDialog)

import numpy as np
import pandas as pd
import scipy.linalg as sp
import sys
import time

In [2]:
#GENERAREA SI CITIREA FISIERLOR

def genereaza_Fisiere(n, nume_matrice, format="csv"):
    # FALLBACK CSV
    extensii = {"csv": ".csv", "xlsx": ".xlsx", "txt": ".txt"}

    if format not in extensii:
        raise ValueError("Formatul specificat nu este acceptat. Folosește 'csv', 'xlsx' sau 'txt'.")

    nume_matrice = f"{nume_matrice}{extensii[format]}"

    matrice = np.random.randint(-10, 10, size=(n, n))
    dfM = pd.DataFrame(matrice)

    if format == "csv":
        dfM.to_csv(nume_matrice, index=False, header=False)
    elif format == "xlsx":
        with pd.ExcelWriter(nume_matrice) as writer:
            dfM.to_excel(writer, index=False, header=False)
    elif format == "txt":
        np.savetxt(nume_matrice, matrice, fmt="%d", delimiter="\t")
    else:
        raise ValueError("Formatul specificat nu este acceptat. Folosește 'csv', 'xlsx' sau 'txt'.")


def genereaza_Matrice_Simetrica_PozitivDefinita(n, nume_matrice, format="csv"):
    extensii = {"csv": ".csv", "xlsx": ".xlsx", "txt": ".txt"}

    if format not in extensii:
        raise ValueError("Formatul specificat nu este acceptat. Folosește 'csv', 'xlsx' sau 'txt'.")

    nume_matrice = f"{nume_matrice}{extensii[format]}"

    matrice = np.random.randint(-10, 10, size=(n, n))
    matrice_sim = (matrice + matrice.T) / 2

    while not np.all(np.linalg.eigvals(matrice_sim) > 0):
        # Dacă matricea nu este pozitiv definita, adaugam un mic termen de corectie la diagonala principala
        matrice_sim += np.eye(n) * 0.1

    dfM = pd.DataFrame(matrice_sim)

    if format == "csv":
        dfM.to_csv(nume_matrice, index=False, header=False)
    elif format == "xlsx":
        with pd.ExcelWriter(nume_matrice) as writer:
            dfM.to_excel(writer, index=False, header=False)
    elif format == "txt":
        np.savetxt(nume_matrice, matrice, fmt="%d", delimiter="\t")
    else:
        raise ValueError("Formatul specificat nu este acceptat. Folosește 'csv', 'xlsx' sau 'txt'.")


def citeste_din_fisier_Matrice(matrice):
    try:
        if matrice.endswith(".csv"):
            
            df = pd.read_csv(matrice, header=None)
        elif matrice.endswith(".xlsx"):
            
            df = pd.read_excel(matrice, header=None)
        elif matrice.endswith(".txt"):
            
            df = pd.read_csv(matrice, sep="\t", header=None)
        else:
            raise ValueError("Formatul specificat nu este acceptat. Folosește 'csv', 'xlsx' sau 'txt'.")

        # Convertim DataFrame-ul într-o listă de liste (matrice)
        data = df.values.tolist()
        matrice = np.reshape(data, (len(df.columns), df.shape[0]))

        x = np.zeros(len(df.columns))  # Se creeaza o matrice goala

        # Verificam daca matricea este patratica
        n = len(matrice)
        for rand in matrice:
            if len(rand) != n:
                raise ValueError("Matricea nu este pătratica!")

        # Folosim adunare aici pentru a face conversia usoara la float
        return x + matrice
    except FileNotFoundError:
        print("Fisierul nu a fost gasit!")


def salveaza_Rezultate(matrice, nume_fisier, format_fisier="csv"):
    extensii = {"csv": ".csv", "xlsx": ".xlsx", "txt": ".txt"}

    if format_fisier not in extensii:
        raise ValueError("Formatul specificat nu este acceptat. Foloseste 'csv', 'xlsx' sau 'txt'.")

    if format_fisier == "csv":
        pd.DataFrame(matrice).to_csv(nume_fisier, index=False, header=False)
    elif format_fisier == "xlsx":
        with pd.ExcelWriter(nume_fisier) as writer:
            pd.DataFrame(matrice).to_excel(writer, index=False, header=False)
    elif format_fisier == "txt":
        np.savetxt(nume_fisier, matrice, fmt="%f", delimiter=" ")

    return f"Matricea a fost salvata cu succes in {nume_fisier}."

In [3]:
#CRITERIILE PENTRU CHOLESKI

def simetrie(matrice):
    return np.allclose(matrice, matrice.T) #Se verifica daca este simetrica
def este_Pozitiv_Definita(matrice):
    valoriProprii = sp.eigvals(matrice) #Se verifica daca este pozitiv definita
    return np.all(valoriProprii > 0)

In [4]:
#DESCOMPUNERE LU "SIMPLA"

def descompunere_lu_directa(matrice):
    n = matrice.shape[0]  
    print(n)
    L = np.zeros((n, n))  
    U = np.zeros((n, n))  

    # Setam prima valoare a lui L si U
    L[0, 0] = 1
    U[0, 0] = matrice[0, 0]
    if L[0, 0] * U[0, 0] == 0:
        print (L[0,0] * U[0, 0])
        print("Factorizare imposibila")
        return None, None

    # Calculam prima linie a lui U si prima coloana a lui L
    for j in range(1, n):
        U[0, j] = matrice[0, j] / L[0, 0]  
        L[j, 0] = matrice[j, 0] / U[0, 0]  

    # Calculam restul valorilor lui L si U
    for i in range(1, n-1):
        L[i, i] = 1  # Setam valoarea diagonala a lui L la 1
        
        U[i, i] = matrice[i, i] - sum(L[i, k] * U[k, i] for k in range(i)) # Calculam elementul diagonal al lui U
        if L[i, i] * U[i, i] == 0:  
            print(L[i, i]*U[i, i])
            print("Factorizare imposibila")
            return None, None

        # Calculam elementele de pe linia i a lui U si coloana i a lui L
        for j in range(i + 1, n):
           
            U[i, j] = (matrice[i, j] - sum(L[i, k] * U[k, j] for k in range(i))) / L[i, i]
           
            L[j, i] = (matrice[j, i] - sum(L[j, k] * U[k, i] for k in range(i))) / U[i, i]

    # Calculam ultima valoare a lui L si U
    L[n-1, n-1] = 1  # Setam ultima valoare diagonala a lui L la 1
    # Calculam ultima valoare diagonala a lui U
    U[n-1, n-1] = matrice[n-1, n-1] - sum(L[n-1, k] * U[k, n-1] for k in range(n-1))
    if L[n-1, n-1] * U[n-1, n-1] == 0:
        return None, None # Nu stiu exact ce trebuie trimis aici

    return L, U

In [5]:
#DESCOMPUNERE LU CHOLESKY

def descompunere_Cholesky(matrice):
    # Verificam daca matricea este simetrica
    if simetrie(matrice) == False:
        raise AssertionError("Matricea nu este simetrica")
    
    # Verificam daca matricea este pozitiv definita
    if este_Pozitiv_Definita(matrice) == False:
        raise AssertionError("Matricea nu este pozitiv definita")
    
    n = matrice.shape[0]
    L = np.zeros((n, n))
    
    # Calculam prima valoare a lui L
    L[0, 0] = np.sqrt(matrice[0, 0])

    # Calculam prima coloana a lui L
    for j in range(1, n):
        L[j, 0] = matrice[j, 0] / L[0, 0]

    # Calculam restul valorilor lui L
    for i in range(1, n):
        suma = sum(L[i, k] ** 2 for k in range(i))
        L[i, i] = np.sqrt(matrice[i, i] - suma)

        for p in range(i + 1, n):
            suma = sum(L[p, k] * L[i, k] for k in range(i))
            L[p, i] = (matrice[p, i] - suma) / L[i, i]

    return L  # U = Transpusa lui L



#L=descompunere_Cholesky(np.array([[6,3,4],[3,6,5],[4,5,10]]))
#U=L.T
#print(" L: \n", L, "\n\n","U: \n", U)

In [6]:
#METODA GAUSS-JORDAN PENTRU CALCULUL MATRICEI INVERSE

def gauss_jordan(matrice, n):
    # Initializam o matrice pentru coeficientii calculati
    coef = np.zeros((n, n))
    
    for i in range(n):
        if matrice[i, i] == 0:  # Verificam daca pivotul este zero
            return None, None
            

        # Normalizam randul curent
        coef[i, i] = 1
        matrice[i] = matrice[i] / matrice[i, i]

        for j in range(n):
            if i != j:
                # Calculam coeficientul pentru randul j
                coef[j, i] = matrice[j, i] / matrice[i, i]
                # Actualizam randul j
                matrice[j] = matrice[j] - matrice[i] * matrice[j, i]
    
    
    inversa = matrice[:, n:]
   
    return inversa, coef


In [7]:
#INTERFATA GRAFICA

matrice_incarcata = None
matrice_inversa = None
L, U=None, None



def verificare_marime_matrice(valoare):
    try:
        if valoare.strip():  # Verificati daca valoarea nu este goala dupa eliminarea spatiilor
            n = int(valoare)
            if n > 100 or n<=0:
                QMessageBox.critical(window, "Atentie!", "Valoarea prea mare, introduceti n pozitiv mai mic sau egal cu 100!")
                generate_button.setEnabled(False)  # Blocheaza butonul daca valoarea este prea mare
                generate_cholesky.setEnabled(False)
                generate_button_cholesky_program.setEnabled(False)
                generate_button_program.setEnabled(False)
            else:
                generate_button.setEnabled(True)   # Deblocheaza butonul daca valoarea este in limite
                generate_cholesky.setEnabled(True)
                generate_button_cholesky_program.setEnabled(True)
                generate_button_program.setEnabled(True)
                
                
        else:
            generate_button.setEnabled(False)  # Blocheaza butonul daca valoarea este goala
            generate_cholesky.setEnabled(False)
            generate_button_cholesky_program.setEnabled(False)
            generate_button_program.setEnabled(False)
            
    except ValueError:
        QMessageBox.critical(window, "Numar invalid!", "Verificati daca ati introdus un numar corect!")
        generate_button.setEnabled(False)
        generate_cholesky.setEnabled(False)
        generate_button_cholesky_program.setEnabled(False)
        generate_button_program.setEnabled(False)
        


def genereaza_fisiere():
    try:
        n = int(size_input.text())
        format = format_combo.currentText()
        nume_matrice = QFileDialog.getSaveFileName(window, "Genereaza matrice", "")[0]
        if not nume_matrice:
            return
        genereaza_Fisiere(n, nume_matrice, format)  
        result_display.setText(f"Fisierul {nume_matrice}.{format} a fost generat cu succes.")
    except Exception as e:
        QMessageBox.critical(window, "Eroare", str(e))


def genereaza_Matrice_Sim_PozDef():
    try:
        n = int(size_input.text())
        format = format_combo.currentText()
        nume_matrice = QFileDialog.getSaveFileName(window, "Genereaza matrice Cholesky", "")[0]
        if not nume_matrice:
            return
        
        genereaza_Matrice_Simetrica_PozitivDefinita(n, nume_matrice, format)  
        
        result_display.setText(f"Fisierul {nume_matrice}.{format} a fost generat cu succes.")
    except Exception as e:
        QMessageBox.critical(window, "Eroare", str(e))
     

def genereaza_Matrice_DinProgram():
    global matrice_incarcata
    try:
       
        n=int(size_input.text())
        matrice_incarcata=np.random.randint(-10,10,(n,n))
        matrice_incarcata_str = np.array2string(matrice_incarcata, precision=4, suppress_small=True)
        matrix_display.setText(str(matrice_incarcata_str))
        lu_button.setEnabled(True)
        inverse_button.setEnabled(True)
        comp_button.setEnabled(True)
        
        
    except Exception as e:
        QMessageBox.critical(window, "Eroare", str(e))
        
def genereaza_MatricePD_DinProgram(): # Genereaza o matrice pozitiv definita
    global matrice_incarcata
    
    n=int(size_input.text())
    matrice=np.random.randint(-10,10, size=(n,n))
    matrice_sim= (matrice+matrice.T)/2
    
    while not np.all(np.linalg.eigvals(matrice_sim) > 0):
        # Daca matricea nu este pozitiv definita, adaugam un mic termen de corectie la diagonala principala
        matrice_sim += np.eye(n) * 0.1
    
    matrice_incarcata=matrice_sim
    matrice_incarcata_str = np.array2string(matrice_incarcata, precision=4, suppress_small=True)
    matrix_display.setText(str(matrice_incarcata_str))
    lu_button.setEnabled(True)
    inverse_button.setEnabled(True)
    comp_button.setEnabled(True)
    

def incarca_matrice():
    global matrice_incarcata
    try:
        nume_fisier, _ = QFileDialog.getOpenFileName(window, "Incarca fisierul cu matricea", "", "Toate fisierele (*);;Fisiere CSV (*.csv);;Fisiere Excel (*.xlsx);;Fisiere text (*.txt)")
        if not nume_fisier:
            return
        matrice_incarcata = citeste_din_fisier_Matrice(nume_fisier)
        matrice_incarcata_str = np.array2string(matrice_incarcata,precision=4,suppress_small=True)
        matrix_display.setText(str(matrice_incarcata_str))
        lu_button.setEnabled(True)
        inverse_button.setEnabled(True)
        comp_button.setEnabled(True)
        
        
    except Exception as e:
        QMessageBox.critical(window, "Eroare", str(e))
        
def incarca_matrice_din_tastatura():
    global matrice_incarcata
    try:
        text, ok = QInputDialog.getMultiLineText(window, "Introdu matricea", "Matricea (elementele separate prin spatiu, randurile separate prin enter):")
        if ok and text:
            rows = text.strip().split('\n')
            matrice = [list(map(float, row.split())) for row in rows]
            matrice_incarcata = np.array(matrice)
            print(matrice_incarcata)
            
            matrice_incarcata_str = np.array2string(matrice_incarcata, precision=4, suppress_small=True)
            matrix_display.setText(str(matrice_incarcata_str))
            
            lu_button.setEnabled(True)
            inverse_button.setEnabled(True)
            comp_button.setEnabled(True)
            
    except Exception as e:
        QMessageBox.critical(window, "Eroare", str(e))


    
def efectueaza_descompunere(matrice):
    global L,U
    try:
        
        if simetrie(matrice) and este_Pozitiv_Definita(matrice):
            QMessageBox.information(window,"Cholesky", "Se va folosi metoda Cholesky deoarece matricea satisface condiitile")
            L = descompunere_Cholesky(matrice)
            U = L.T
            
            L_str = np.array2string(L, precision=4, suppress_small=True)
            U_str = np.array2string(U, precision=4, suppress_small=True)
            
            verificare_button_LU.setEnabled(True)
            result_display.setText(f"L:\n{L_str}\n\nU:\n{U_str}")
            nume_fisier = QFileDialog.getSaveFileName(window, "Salveaza fisierul L", "", f"Fisier {format_combo.currentText().upper()} (*.{format_combo.currentText()})")[0]
            salveaza_Rezultate(L, nume_fisier, format_combo.currentText())
            nume_fisier = QFileDialog.getSaveFileName(window, "Salveaza fisierul U", "", f"Fisier {format_combo.currentText().upper()} (*.{format_combo.currentText()})")[0]
            salveaza_Rezultate(U, nume_fisier, format_combo.currentText())
            
        else:
            L, U = descompunere_lu_directa(matrice)
            if L is None and U is None:
                QMessageBox.critical(window, "Eroare", "Factorizare imposibila")
                return
                
            QMessageBox.information(window,"Direct", "Se va folosi metoda Directa deoarece matricea nu este simetrica si pozitiv definita")
            
            L_str = np.array2string(L, precision=4, suppress_small=True)
            U_str = np.array2string(U, precision=4, suppress_small=True)
            
            verificare_button_LU.setEnabled(True)
            result_display.setText(f"L:\n{L_str}\n\nU:\n{U_str}")
            nume_fisier = QFileDialog.getSaveFileName(window, "Salveaza fisierul L", "", f"Fisier {format_combo.currentText().upper()} (*.{format_combo.currentText()})")[0]
            salveaza_Rezultate(L, nume_fisier, format_combo.currentText())
            nume_fisier = QFileDialog.getSaveFileName(window, "Salveaza fisierul U", "", f"Fisier {format_combo.currentText().upper()} (*.{format_combo.currentText()})")[0]
            salveaza_Rezultate(U, nume_fisier, format_combo.currentText())
            
    
    except Exception as e:
        #QMessageBox.critical(window, "Error", str(e))
        return
        
def comparare_timp_metode(matrice):
    try:
        if simetrie(matrice) and este_Pozitiv_Definita(matrice):
            
            start = time.time()
            descompunere_lu_directa(matrice)
            end = time.time()
            timp_lu_directa = end - start

            start = time.time()
            descompunere_Cholesky(matrice)
            end = time.time()
            timp_cholesky = end - start
            
            message = f"Timpul de executie LU Directa: {timp_lu_directa} secunde \n Timpul de executie LU Cholesky: {timp_cholesky} secunde"
            
            QMessageBox.information(window, "Comparare timp metode", message)
        
        else:
            QMessageBox.critical(window, "Atentie!", "Matricea nu poate fi descompusa in mod Cholesky")
        
        
    
    except Exception as e:
        #QMessageBox.critical(window, "Eroare", str(e))
        return
        

def calculeaza_inversa(matrice):
    global matrice_inversa
    try:
        
        n = len(matrice)
        matrice_augmentata = np.hstack((matrice, np.eye(n)))
        inversa, coef = gauss_jordan(matrice_augmentata, n)
        if inversa is None:
           QMessageBox.critical(window, "Eroare", "Matricea are 0 pe diagonala principala sau determinantul este egal cu 0")
           return
        
        inversa_str = np.array2string(inversa, precision=4, suppress_small=True)
        coef_str = np.array2string(coef, precision=4, suppress_small=True)
        matrice_inversa = inversa
        
        result_display.setText(f"Matricea inversa:\n{inversa_str} \n\n Coeficientii: \n{coef_str}")
        verificare_button.setEnabled(True)
        nume_fisier = QFileDialog.getSaveFileName(window, "Salveaza fisierul inversa matricei", "", f"Fisier {format_combo.currentText().upper()} (*.{format_combo.currentText()})")[0]
        salveaza_Rezultate(inversa, nume_fisier, format_combo.currentText())
        nume_fisier_coef = QFileDialog.getSaveFileName(window, "Salveaza fisierul coeficienti", "", f"Fisier {format_combo.currentText().upper()} (*.{format_combo.currentText()})")[0]
        salveaza_Rezultate(coef, nume_fisier_coef, format_combo.currentText())
    except Exception as e:
        #QMessageBox.critical(window, "Eroare", str(e))
        return

def verifica_Inversa(matrice_inversa, matrice_incarcata):
    n=matrice_inversa.shape[0]
    verificare = np.dot(matrice_inversa, matrice_incarcata)
    
    matrice_inversa_str = np.array2string(matrice_inversa, precision=4, suppress_small=True)
    matrice_incarcata_str= np.array2string(matrice_incarcata, precision=4,suppress_small=True)
    verificare_str = np.array2string(verificare, precision=4, suppress_small=True)
    if np.allclose(verificare, np.eye(n)):
        
        QMessageBox.information(window,"Verificare inversa", f"Matricea inversa este corecta: \n  {matrice_inversa_str}\n*\n{matrice_incarcata_str}\n = \n{verificare_str}")
    else:
        print(np.dot(matrice_inversa, matrice_incarcata))
        QMessageBox.critical(window, "Verificare inversa", "Matricea inversa este incorecta, incercati sa calculati din nou")

def verifica_LU(lower, upper):
    n=lower.shape[0]
    verificare = np.dot(lower,upper)
    lower_str=np.array2string(lower, precision=4, suppress_small=True)
    upper_str=np.array2string(upper, precision=4, suppress_small=True)
    verificare_str=np.array2string(verificare, precision=4, suppress_small=True)
    
    
    if np.allclose(verificare, matrice_incarcata):
        QMessageBox.information(window,"Verficare LU", f"Matricile LU sunt corecte: \n{upper_str}\n*\n{lower_str}\n = \n{verificare_str}")
    else:
        QMessageBox.critical(window,"Verificare LU", "Matricile LU sunt incorecte, incercati sa calculati din nou")
        
    

app = QApplication(sys.argv)
window = QWidget()
window.setWindowTitle("Proiect Calcul Numeric")

# Layout-uri
main_layout = QVBoxLayout()

# Intrare pentru dimensiunea matricei si formatul fisierului
input_layout = QHBoxLayout()
size_input = QLineEdit('3')
format_combo = QComboBox()
format_combo.addItems(["csv", "xlsx", "txt"])
input_layout.addWidget(QLabel("Dimensiune:"))
input_layout.addWidget(size_input)
input_layout.addWidget(QLabel("Format:"))
input_layout.addWidget(format_combo)
generate_button = QPushButton("Genereaza Matrice")
generate_cholesky = QPushButton("Genereaza M.Cholesky")
generate_button_program = QPushButton("Genereaza M")
generate_button_cholesky_program = QPushButton("Genereaza MC")
generate_button.clicked.connect(genereaza_fisiere)
generate_cholesky.clicked.connect(genereaza_Matrice_Sim_PozDef)
generate_button_program.clicked.connect(genereaza_Matrice_DinProgram)
generate_button_cholesky_program.clicked.connect(genereaza_MatricePD_DinProgram)
input_layout.addWidget(generate_button)
input_layout.addWidget(generate_cholesky)
input_layout.addWidget(generate_button_program)
input_layout.addWidget(generate_button_cholesky_program)
main_layout.addLayout(input_layout)

# Intrare pentru matrice 
file_layout = QHBoxLayout()
load_matrix_button = QPushButton("Incarca Fisier")
load_matrix_button.clicked.connect(incarca_matrice)
file_layout.addWidget(load_matrix_button)
load_matrix_keyboard_button = QPushButton("Incarca de la Tastatura")
load_matrix_keyboard_button.clicked.connect(incarca_matrice_din_tastatura)
file_layout.addWidget(load_matrix_keyboard_button)
main_layout.addLayout(file_layout)


# Afisare matrice si solutii
matrix_display = QTextEdit()
main_layout.addWidget(QLabel("Matrice:"))
main_layout.addWidget(matrix_display)

# Butoane pentru operatii
operation_layout = QHBoxLayout()
lu_button = QPushButton("Descompunere LU")
lu_button.clicked.connect(lambda: efectueaza_descompunere(matrice_incarcata) if matrice_incarcata is not None else QMessageBox.warning(window, "Atentie", "Nu a fost incarcata nicio matrice."))

operation_layout.addWidget(lu_button)

inverse_button = QPushButton("Calculeaza Inversa")
inverse_button.clicked.connect(lambda: calculeaza_inversa(matrice_incarcata) if matrice_incarcata is not None else QMessageBox.warning(window, "Atentie", "Nu a fost incarcata nicio matrice."))
operation_layout.addWidget(inverse_button)
main_layout.addLayout(operation_layout)

# Buton pentru comparare de timp
comp_timp_layout = QHBoxLayout()
comp_button = QPushButton("Compararea Timp")
comp_button.clicked.connect(lambda: comparare_timp_metode(matrice_incarcata) if matrice_incarcata is not None else QMessageBox.warning(window, "Atentie", "Nu a fost incarcata nicio matrice."))
comp_timp_layout.addWidget(comp_button)
main_layout.addLayout(comp_timp_layout)

# Buton pentru verificare inversa
verificare_button = QPushButton("Verificare Inversa")
verificare_button.clicked.connect(lambda: verifica_Inversa(matrice_incarcata, matrice_inversa) if matrice_inversa is not None else QMessageBox.warning(window, "Atentie", "Nu exista o matrice inversa"))
comp_timp_layout.addWidget(verificare_button)

#Buton pentru verificare LU
verificare_button_LU = QPushButton("Verificare LU")
verificare_button_LU.clicked.connect(lambda: verifica_LU(L,U) if L is not None and U is not None else QMessageBox.warning(window, "Atentie", "Nu exista valori L U"))
comp_timp_layout.addWidget(verificare_button_LU)

lu_button.setEnabled(False)
comp_button.setEnabled(False)
inverse_button.setEnabled(False)
verificare_button.setEnabled(False)
verificare_button_LU.setEnabled(False)

size_input.textChanged.connect(verificare_marime_matrice)

# Afisare rezultate
result_display = QTextEdit()
result_display.setReadOnly(True)
matrix_display.setReadOnly(True)
main_layout.addWidget(QLabel("Rezultat:"))
main_layout.addWidget(result_display)
window.setLayout(main_layout)
window.show()
sys.exit(app.exec_())


SystemExit: 0

c:\Users\Windows Power User 2\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
